In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
from collections import Counter

from tqdm import tqdm
import azure.cosmos.cosmos_client as cosmos_client

# Useful queries

Example queries against the document DB instance.

Cosmos DB Python SDK documentation: https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python

Cosmos DB Resource URI Syntax for REST: https://docs.microsoft.com/en-us/rest/api/cosmos-db/cosmosdb-resource-uri-syntax-for-rest

## Connect to the Cosmos DB instance

`COSMOS_ENDPOINT` and `COSMOS_KEY` need to be environment variables. 

In [6]:
# Cosmos DB config
config = {
    'ENDPOINT': os.environ.get('COSMOS_ENDPOINT'),
    'PRIMARYKEY': os.environ.get('COSMOS_KEY')
}

# Initialize the Cosmos client
client = cosmos_client.CosmosClient(url_connection=config['ENDPOINT'], auth={
                                    'masterKey': config['PRIMARYKEY']})

container_link = 'dbs/camera-trap/colls/images'  # database link + container link

## Examples

In the examples, we limit the selection to a few entries using the `TOP` keyword. When using the DB to create datasets, delete the TOP keyword and arg.

In [89]:
dataset = 'rspb_gola'

### Image entries in a specified dataset
by specifying the `partition_key`, which we use the `dataset` attribute for.

Avoid doing this on more than a couple thousands items (use TOP) - should probably get the entry id and go from there.

In [90]:
%%time

query = {'query': '''
SELECT TOP 1234 im
FROM images im
'''}

options = {
    'enableCrossPartitionQuery': True
}

result_iterable = client.QueryItems(container_link, query, options, partition_key=dataset)
# if you want to restrict to one dataset, pass in partition_key=dataset

results = []
for item in iter(result_iterable):
    res = item['im']
    results.append(res)

print('Length of results:', len(results))

Length of results: 1234
CPU times: user 179 ms, sys: 64.3 ms, total: 243 ms
Wall time: 4.11 s


### All images with bounding box annotation

In [26]:
%%time

query = {'query': '''
SELECT TOP 1020 im
FROM images im
WHERE ARRAY_LENGTH(im.annotations.bbox) > 0
'''}

options = {
    'enableCrossPartitionQuery': True
}

result_iterable = client.QueryItems(container_link, query, options)
# if you want to restrict to one dataset, pass in partition_key=dataset

results = []
for item in iter(result_iterable):
    res = item['im']
    results.append(res)

print('Length of results:', len(results))

Length of results: 1020
CPU times: user 115 ms, sys: 9.8 ms, total: 125 ms
Wall time: 1.44 s


### All images with the specified species

Top 1,000 docs return in about 3s. 

Top 6,000 docs return in about 10s.

In [33]:
%%time

species_requested = 'human'

query = {'query': '''
SELECT TOP 100000 im
FROM images im
WHERE ARRAY_LENGTH(im.annotations.species) > 0 AND ARRAY_CONTAINS(im.annotations.species, "{}")
'''.format(species_requested)}

options = {
    'enableCrossPartitionQuery': True
}

result_iterable = client.QueryItems(container_link, query, options)

results = []
for item in iter(result_iterable):
    res = item['im']
    results.append(res)
    
print('Length of results:', len(results))

Length of results: 6444
CPU times: user 515 ms, sys: 41.9 ms, total: 557 ms
Wall time: 9.25 s


### Species count

In [29]:
%%time

query = {'query': '''
SELECT TOP 1020 im.annotations.species
FROM images im
WHERE ARRAY_LENGTH(im.annotations.species) > 0
'''}

options = {
    'enableCrossPartitionQuery': True
}

result_iterable = client.QueryItems(container_link, query, options)

species = Counter()
for item in iter(result_iterable):
    res = item['species']
    species.update(res)

CPU times: user 92.1 ms, sys: 7.24 ms, total: 99.3 ms
Wall time: 1.64 s


In [30]:
species

Counter({'Cattle': 3,
         'Moose': 1,
         'deer': 188,
         'elk': 156,
         'empty': 669,
         'human': 4})

### Total number of image entries in database

In [79]:
query = {'query': '''
SELECT VALUE COUNT(1)
FROM images
'''}

options = {
    'enableCrossPartitionQuery': True
}

result_iterable = client.QueryItems(container_link, query, options)

In [80]:
for count in iter(result_iterable):
    print(count)

702479


### List last inserted items based on insertion timestap

In [73]:
%%time

query = {'query': '''
SELECT TOP 16164 im
FROM images im
ORDER BY im._ts DESC
'''}

options = {
    'enableCrossPartitionQuery': True
}

result_iterable = client.QueryItems(container_link, query, options)

results = []
for item in iter(result_iterable):
    res = item['im']
    results.append(res)

CPU times: user 1.54 s, sys: 110 ms, total: 1.65 s
Wall time: 19.8 s


In [74]:
len(results)

16164